# Split datasets


In [2]:
import json
import os
import shutil
import pandas as pd
import requests
from sklearn.model_selection import train_test_split

In [3]:
# label_map_csv = "../datasets/cassava-leaf-disease-classification/train.csv"
# label_map_csv = "../datasets/PAD-UFES-20/metadata.csv"
label_map_csv = "../datasets/ddi-diverse-dermatology-images/ddi_metadata.csv"
images_root = "../datasets/ddi-diverse-dermatology-images/"
split_column = "malignant"
file_column = "DDI_file"
image_type = ""

label_map_csv = "../datasets/ddi-diverse-dermatology-images/ddi_metadata.csv"
images_root = "../datasets/ddi-diverse-dermatology-images/"
split_column = "disease"
file_column = "DDI_file"
image_type = ""

label_map_csv = "../datasets/HAM10000/HAM10000_metadata"
images_root = "../datasets/HAM10000/images/"
split_column = "dx"
file_column = "image_id"
image_type = ".jpg"

label_map_csv = "../datasets/PAD-UFES-20/metadata.csv"
images_root = "../datasets/PAD-UFES-20/images/"
split_column = "diagnostic"
file_column = "img_id"
image_type = ""

label_map_csv = "../datasets/fitzpatrick17k/fitzpatrick17k.csv"
images_root = "../datasets/fitzpatrick17k/images/"
split_column = "nine_partition_label"
file_column = "url"
image_type = ""
# df_full["qc"].unique() -> array([nan, '1 Diagnostic', '5 Potentially', '3 Wrongly labelled', '2 Characteristic', '4 Other'], dtype=object)

In [4]:
df_full = pd.read_csv(label_map_csv)
df_full.columns

Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'label',
       'nine_partition_label', 'three_partition_label', 'qc', 'url',
       'url_alphanum'],
      dtype='object')

In [17]:
for image_url in df_full["url"]:
    file_name = None
    if str(image_url) == "nan":
        # print(f"image_url: {image_url}")
        # print(f"file_name: {file_name}")
        continue
    elif str(image_url).startswith("https://www.dermaamin.com"):
        continue
    elif str(image_url).startswith("http://atlasdermatologico.com.br/img"):
        continue

    file_name = os.path.basename(image_url)
    file_name = file_name.replace("?", "")
    file_path = os.path.join(images_root, file_name)

    if not os.path.exists(file_path):
        response = requests.get(image_url, stream=True, headers={"User-Agent": "XY"})
        if not response.ok:
            print(f"image_url: {image_url}")
            print(response)
            continue

        with open(file_path, "wb") as handle:
            for block in response.iter_content(1024):
                if not block:
                    break
                handle.write(block)

In [19]:
for value in df_full[split_column].unique():
    subdirectory = os.path.join(images_root, split_column, str(value))
    print(f"Copying to {subdirectory}")
    os.makedirs(subdirectory, exist_ok=True)
    for image_path in df_full[df_full[split_column] == value][file_column]:
        if(str(image_path) == "nan"):
            continue

        image_name = os.path.basename(image_path)
        image_name = image_name.replace("?", "")
        image_name += image_type
        source_path = os.path.join(images_root, image_name)

        if os.path.exists(source_path):
            shutil.copy(source_path, f"{subdirectory}/")


Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\inflammatory
Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\benign dermal
Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\malignant dermal
Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\malignant melanoma
Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\genodermatoses
Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\malignant epidermal
Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\benign epidermal
Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\benign melanocyte
Copying to ../datasets/fitzpatrick17k/images/nine_partition_label\malignant cutaneous lymphoma


In [26]:
# df_train, df_test = train_test_split(
#     df_full, test_size=test_ratio, stratify=df_full["label"], random_state=seed
# )
# print(df_train["label"].value_counts())
# print(df_test["label"].value_counts())

# _, df_train_mini = train_test_split(
#     df_train, test_size=0.1, stratify=df_train["label"], random_state=seed
# )
# print(df_train_mini["label"].value_counts())

In [27]:
# entries = json.load(
#     open(
#         "../datasets/cassava-leaf-disease-classification/label_num_to_disease_map.json"
#     )
# )
# entries

In [28]:
# for key in entries:
#     value = entries[key]
#     os.makedirs(f"../datasets/cassava-dataset/train/{value}/", exist_ok=True)
#     os.makedirs(f"../datasets/cassava-dataset/test/{value}/", exist_ok=True)
#     os.makedirs(f"../datasets/cassava-dataset/train-mini/{value}/", exist_ok=True)

In [29]:
# for index, row in df_test.iterrows():
#     image_id = row["image_id"]
#     label_key = row["label"]
#     shutil.copy(
#         f"../datasets/cassava-leaf-disease-classification/train_images/{image_id}",
#         f"../datasets/cassava-dataset/test/{entries[str(label_key)]}/"
#     )

In [30]:
# for index, row in df_train.iterrows():
#     image_id = row["image_id"]
#     label_key = row["label"]
#     shutil.copy(
#         f"../datasets/cassava-leaf-disease-classification/train_images/{image_id}",
#         f"../datasets/cassava-dataset/train/{entries[str(label_key)]}/"
#     )

In [31]:
# for index, row in df_train_mini.iterrows():
#     image_id = row["image_id"]
#     label_key = row["label"]
#     shutil.copy(
#         f"../datasets/cassava-leaf-disease-classification/train_images/{image_id}",
#         f"../datasets/cassava-dataset/train-mini/{entries[str(label_key)]}/"
#     )